#### web crawler code
urls and metadata was obtained manually and loaded into a spreadsheet news_list and stored in group google drive


In [16]:
import pandas as pd

# Define the path to your Excel file
#file_path = r'G:\My Drive\699 capstone\news\news_list.xlsx' 
file_path = r'G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\news\news_list.xlsx'

# Load the Excel file into a DataFrame
news_df = pd.read_excel(file_path)
#df= news_df
# Display the first few rows to confirm the data loaded correctly
#df = news_df[news_df["pdf"] != "yes"]  & (news_df["url"].notna())]
#df = news_df[(news_df["pdf"] != "yes") & (news_df["url"].notna())]
df = news_df[(news_df["pdf"] != "yes") & (news_df["url"].notna()) & (news_df["ticker"] == "TSLA")]
# Display the first few rows to confirm the filtering
df.head(50)


,ticker,pdf,date,type,source,url,title,Unnamed: 7,date_name
80,TSLA,NaN,2024-10-02,Company presentation,Tesla,https://digitalassets.tesla.com/tesla-contents...,TSLA 2024 Q3 Quarterly Update Deck,2024-10-02,2024-10-02_TSLA_Company presentation_TSLA 2024...
81,TSLA,NaN,2024-10-02,Press release,Tesla,https://ir.tesla.com/press-release/tesla-third...,"Tesla Third Quarter 2024 Production, Deliverie...",2024-10-02,2024-10-02_TSLA_Press release_Tesla Third Quar...


In [17]:
import time
import base64
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import os
import requests
import pandas as pd

def save_url_as_pdf(url, output_folder, filename):
    # Check if the URL contains the word "pdf"
    if "pdf" in url.lower():
        # Define the output path for the PDF in the specified folder
        output_path = os.path.join(output_folder, f'{filename}.pdf')
        
        # Download the PDF directly from the URL
        response = requests.get(url)
        if response.status_code == 200:
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"PDF saved directly at {output_path}")
        else:
            print(f"Failed to download PDF from {url}")
        return

    # Set up Chrome options for headless mode (for non-PDF URLs)
    chrome_options = Options()
    chrome_options.add_argument('--headless=new')  # Use the new headless mode if available
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    
    # Path to ChromeDriver
    chrome_service = Service("C:\\Windows\\chromedriver-win64\\chromedriver.exe")

    # Initialize the WebDriver
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    
    try:
        # Open the URL
        driver.get(url)
        time.sleep(3)  # Allow some time for the page to fully load
        
        # Use Chrome DevTools Protocol to save the page as a PDF
        pdf_data = driver.execute_cdp_cmd("Page.printToPDF", {
            "printBackground": True  # Prints background graphics if necessary
        })

        # Define the output path for the PDF in the specified folder
        output_path = os.path.join(output_folder, f'{filename}.pdf')

        # Decode the base64 PDF data and save it as a binary file
        with open(output_path, "wb") as f:
            f.write(base64.b64decode(pdf_data['data']))

        print(f"PDF saved at {output_path}")

    except Exception as e:
        print(f"Failed to save {url} as PDF. Error: {e}")
    finally:
        driver.quit()

# # Main function to iterate over the DataFrame and save each URL as a PDF
# def save_all_urls_as_pdfs(dataframe, output_folder):
#     # Ensure the output folder exists
#     os.makedirs(output_folder, exist_ok=True)
    
#     for index, row in dataframe.iterrows():
#         url = row['url']
#         title = row['title']
        
#         if pd.notna(url):  # Ensure URL is not NaN
#             print(f"Processing: {url}")
#             save_url_as_pdf(url, output_folder, title)
        
# # Example usage
# output_folder = './tesla'  # Folder to save PDFs
# save_all_urls_as_pdfs(df, output_folder)


In [18]:
# Main function to iterate over the DataFrame and save each URL as a PDF
def save_all_urls_as_pdfs(dataframe, file_path):
    # Ensure the output folder exists
    os.makedirs(file_path, exist_ok=True)
    
    for index, row in dataframe.iterrows():
        url = row['url']
        title = row['title']
        
        if pd.notna(url):  # Ensure URL is not NaN
            print(f"Processing: {url}")
            save_url_as_pdf(url, file_path, title)
        
# Example usage
file_path = r'G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\news\Tesla'  # Folder to save PDFs
save_all_urls_as_pdfs(df, file_path)

Processing: https://digitalassets.tesla.com/tesla-contents/image/upload/IR/TSLA-Q3-2024-Update.pdf
PDF saved directly at G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\news\Tesla\TSLA 2024 Q3 Quarterly Update Deck.pdf
Processing: https://ir.tesla.com/press-release/tesla-third-quarter-2024-production-deliveries-and-deployments
PDF saved at G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\news\Tesla\Tesla Third Quarter 2024 Production, Deliveries & Deployments.pdf
